# 패키지 설치

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경 변수를 불러옵니다.

# api_key = os.getenv("OPENAI_API_KEY")
# print(f"API Key: {api_key}")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [4]:
# print('hello world')
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores.faiss import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

prompt = PromptTemplate.from_template("Question: {question}\nAnswer:")

Question = '당신은 누구십니까?'

response = llm.invoke(prompt.format(question=Question))

In [6]:
response

AIMessage(content='저는 인공지능 챗봇입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 25, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9debeaa5-75c2-45bb-8ed4-09040f587873-0', usage_metadata={'input_tokens': 25, 'output_tokens': 15, 'total_tokens': 40})